**Name:** SHUYUE JIA

**EID:** 56846018

# CS5489 - Tutorial 4
## Face Detection in Images

In this tutorial you will train a classifier to detect whether there is a face in a small image patch.  This type of face detector is used in your phone and camera whenever you take a picture!

First we need to initialize Python.  Run the below cell.

In [1]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
import os
import zipfile
import fnmatch
random.seed(100)
from scipy import ndimage
from scipy import signal
from scipy import stats
import skimage.color
import skimage.exposure
import skimage.io
import skimage.util
import xgboost as xgb

## 1. Loading Data and Pre-processing
Next we need to load the images.  Download `faces.zip`, and put it in the same direcotry as this ipynb file.  **Do not unzip the file.** Then run the following cell to load the images.

In [2]:
imgdata = {'train':[], 'test':[]}
classes = {'train':[], 'test':[]}

# the dataset is too big, so subsample the training and test sets...
# reduce training set by a factor of 4
train_subsample = 4  
train_counter = [0, 0]
# maximum number of samples in each class for test set
test_maxsample = 472
test_counter = [0, 0]

# load the zip file
filename = 'faces.zip'
zfile = zipfile.ZipFile(filename, 'r')

for name in zfile.namelist():
    # check file name matches
    if fnmatch.fnmatch(name, "faces/*/*/*.png"):
        
        # filename is : faces/train/face/fname.png
        (fdir1, fname)  = os.path.split(name)     # get file name
        (fdir2, fclass) = os.path.split(fdir1) # get class (face, nonface)
        (fdir3, fset)   = os.path.split(fdir2) # get training/test set
        
        # class 1 = face; class 0 = non-face
        myclass = int(fclass == "face")  

        loadme = False
        if fset == 'train':
            if (train_counter[myclass] % train_subsample) == 0:
                loadme = True
            train_counter[myclass] += 1
            
        elif fset == 'test':
            if test_counter[myclass] < test_maxsample:
                loadme = True
            test_counter[myclass] += 1
            
        if (loadme):
            # open file in memory, and parse as an image
            myfile = zfile.open(name)
            
            img = matplotlib.image.imread(myfile)
            # img = skimage.io.imread(myfile)
            # convert to grayscale
            img = skimage.color.rgb2gray(img)
            myfile.close()
            
            # append data
            imgdata[fset].append(img)
            classes[fset].append(myclass)

        
zfile.close()
imgsize = img.shape

print(len(imgdata['train']))
print(len(imgdata['test']))
trainclass2start = sum(classes['train'])


UnsupportedOperation: seek

Each image is a 19x19 array of pixel values.  Run the below code to show an example:

In [ ]:
print(img.shape)
plt.subplot(1,2,1)
plt.imshow(imgdata['train'][0], cmap='gray', interpolation='nearest')
plt.title("face sample")
plt.subplot(1,2,2)
plt.imshow(imgdata['train'][trainclass2start], cmap='gray', interpolation='nearest')
plt.title("non-face sample")
plt.show()

Run the below code to show more images!

In [ ]:
# function to make an image montage
def image_montage(X, imsize=None, maxw=10):
    """X can be a list of images, or a matrix of vectorized images.
      Specify imsize when X is a matrix."""
    tmp = []
    numimgs = len(X)
    
    # create a list of images (reshape if necessary)
    for i in range(0,numimgs):
        if imsize != None:
            tmp.append(X[i].reshape(imsize))
        else:
            tmp.append(X[i])
    
    # add blanks
    if (numimgs > maxw) and (mod(numimgs, maxw) > 0):
        leftover = maxw - mod(numimgs, maxw)
        meanimg = 0.5*(X[0].max()+X[0].min())
        for i in range(0,leftover):
            tmp.append(ones(tmp[0].shape)*meanimg)
    
    # make the montage
    tmp2 = []
    for i in range(0,len(tmp),maxw):
        tmp2.append( hstack(tmp[i:i+maxw]) )
    montimg = vstack(tmp2) 
    return montimg

# show a few images
plt.figure(figsize=(9,9))
plt.imshow(image_montage(imgdata['train'][::20]), cmap='gray', interpolation='nearest')
plt.show()

Each image is a 2d array, but the classifier algorithms work on 1d vectors. Run the following code to convert all the images into 1d vectors by flattening.  The result should be a matrix where each row is a flattened image.

In [ ]:
trainX = empty((len(imgdata['train']), prod(imgsize)))
for i,img in enumerate(imgdata['train']):
    trainX[i,:] = ravel(img)
trainY = asarray(classes['train'])  # convert list to numpy array
print(trainX.shape)
print(trainY.shape)

testX = empty((len(imgdata['test']), prod(imgsize)))
for i,img in enumerate(imgdata['test']):
    testX[i,:] = ravel(img)
testY = asarray(classes['test'])  # convert list to numpy array
print(testX.shape)
print(testY.shape)


## 2. Detection using  pixel values

Train an AdaBoost and GradientBoosting classifiers to classify an image patch as face or non-face.  Also train a kernel SVM classifier using either RBF or polynomial kernel, and a Random Forest Classifier.  Evaluate all your classifiers on the test set.

First we will normalize the features.

In [ ]:
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))    # make scaling object
trainXn = scaler.fit_transform(trainX)   # use training data to fit scaling parameters
testXn  = scaler.transform(testX)        # apply scaling to test data


In [ ]:
# AdaBoost Model
# setup the list of parameters to try
paramgrid = {'learning_rate': logspace(-6, 0, 3)} 
# print(paramgrid)

# setup the cross-validation object 
adaclf = ensemble.AdaBoostClassifier(random_state=4487)
adacv = model_selection.GridSearchCV(adaclf, paramgrid, cv=5, n_jobs=8) 

adacv.fit(trainX, trainY)
# print("best params:", adacv.best_params_)

# predict from the model 
train_pre = adacv.predict(trainX)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(AdaBoost Model)\n training accuracy =", train_acc)

# predict from the model 
test_pre = adacv.predict(testX)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(AdaBoost Model)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


In [ ]:
# GradientBoosting Model
# use the XGBoost package, compatible with sklearn 
import xgboost as xgb

# use "multi:softprob" for multi-class classification 
xclf = xgb.XGBClassifier(objective="binary:logistic", random_state=4487)

# setup the list of parameters to try

paramgrid = {'learning_rate': logspace(-6, 0, 3)} 
print(paramgrid)

# setup the cross-validation object 
xgbcv = model_selection.GridSearchCV(xclf, paramgrid, cv=5, n_jobs=8)

# run cross-validation (train for each split) 
xgbcv.fit(trainX, trainY)
# print("best params:", xgbcv.best_params_)

# predict from the model 
train_pre = xgbcv.predict(trainX)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(GradientBoosting Model)\n training accuracy =", train_acc)

# predict from the model 
test_pre = xgbcv.predict(testX)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(GradientBoosting Model)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


In [ ]:
# GradientBoosting Model
# use the XGBoost package, compatible with sklearn 
import xgboost as xgb

# setup dictionary of distributions for each parameter
paramsampler = {    
    "colsample_bytree": stats.uniform(0.7, 0.3),  # default=1
#     "gamma":            stats.uniform(0, 0.5),    # default=0
#     "max_depth":        stats.randint(2, 6),      # default=6
#     "subsample":        stats.uniform(0.6, 0.4),  # default=1
#     "learning_rate":    stats.uniform(.001, 1),    # default=1 (could also use loguniform)
#     "n_estimators":     stats.randint(10, 1000),
}

xgbclf = xgb.XGBClassifier(objective="binary:logistic", random_state=4487)

# cross-validation via random search
xgbrcv = model_selection.RandomizedSearchCV(xgbclf, 
                                            param_distributions=paramsampler, 
                                            random_state=4487, 
                                            n_iter=100, 
                                            cv=5, 
                                            verbose=1, 
                                            n_jobs=8)


# run cross-validation (train for each split) 
xgbrcv.fit(trainX, trainY)
print("best params:", xgbrcv.best_params_)

# predict from the model 
train_pre = xgbrcv.predict(trainX)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(GradientBoosting Model with more search parameters)\n training accuracy =", train_acc)

# predict from the model 
test_pre = xgbrcv.predict(testX)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(GradientBoosting Model with more search parameters)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


In [ ]:
# SVM Model with RBF kernel
# setup the list of parameters to try 
paramgrid = {'C': logspace(-2, 3, 3), 
             'gamma': logspace(-4, 3, 3)}
print(paramgrid)

# setup the cross-validation object 
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds 
svmrbf = svm.SVC(kernel='rbf')
svmrbfcv = model_selection.GridSearchCV(svmrbf, 
                                     paramgrid, 
                                     cv=5, 
                                     n_jobs=8, 
                                     verbose=True)

# run cross-validation (train for each split) 
svmrbfcv.fit(trainX, trainY);
print("best params:", svmcv.best_params_)

# predict from the model 
train_pre = svmrbfcv.predict(trainX)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(SVM Model with RBF kernel with more search parameters)\n training accuracy =", train_acc)

# predict from the model 
test_pre = svmrbfcv.predict(testX)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(SVM Model with RBF kernel with more search parameters)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


In [ ]:
# SVM Model with polynomial kernel
# setup the list of parameters to try 
paramgrid = {'C': logspace(-2, 3, 3), 
             'gamma': logspace(-4, 3, 3)}
print(paramgrid)

# setup the cross-validation object # pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds 
svmpol = svm.SVC(kernel='poly')
svmpolcv = model_selection.GridSearchCV(svmpol, 
                                     paramgrid, 
                                     cv=5, 
                                     n_jobs=8, 
                                     verbose=True)

# run cross-validation (train for each split) 
svmpolcv.fit(trainX, trainY);
print("best params:", svmcv.best_params_)

# predict from the model 
train_pre = svmpolcv.predict(trainX)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(SVM Model with polynomial kernel with more search parameters)\n training accuracy =", train_acc)

# predict from the model 
test_pre = svmpolcv.predict(testX)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(SVM Model with polynomial kernel with more search parameters)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


In [ ]:
# Random Forest Model
from sklearn import ensemble

# setup the list of parameters to try
paramsampler = {#'max_features': stats.uniform(0,1.0),
                 'max_depth':         stats.randint(1, 5),
                 'min_samples_split': stats.uniform(0, 0.5), 
                 'min_samples_leaf':  stats.uniform(0, 0.5)}

rf = ensemble.RandomForestClassifier(n_estimators=100, random_state=4487, n_jobs=8)

# setup the cross-validation object
rfrcv = model_selection.RandomizedSearchCV(
    rf,
    param_distributions=paramsampler, 
    random_state=4487, 
    n_iter=100, 
    cv=5, 
    verbose=1, 
    n_jobs=8)

# run cross-validation (train for each split)
rfrcv.fit(trainX, trainY);
print("best params:", rfrcv.best_params_)

# predict from the model 
train_pre = rfrcv.predict(trainX)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(Random Forest Model with more search parameters)\n training accuracy =", train_acc)

# predict from the model 
test_pre = rfrcv.predict(testX)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(Random Forest Model with more search parameters)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


_Which classifier was best?_
- SVM Model with RBF kernel according to the results

### Error analysis
The accuracy only tells part of the classifier's performance. We can also look at the different types of errors that the classifier makes:
- _True Positive (TP)_: classifier correctly said face
- _True Negative (TN)_: classifier correctly said non-face
- _False Positive (FP)_: classifier said face, but not a face
- _False Negative (FN)_: classifier said non-face, but was a face

This is summarized in the following table:

<table>
<tr><th colspan=2 rowspan=2><th colspan=2 style="text-align: center">Actual</th></tr>
<tr>  <th>Face</th><th>Non-face</th></tr>
<tr><th rowspan=2>Prediction</th><th>Face</th><td>True Positive (TP)</td><td>False Positive (FP)</td></tr>
<tr>  <th>Non-face</th><td>False Negative (FN)</td><td>True Negative (TN)</td></tr>
</table>

We can then look at the _true positive rate_ and the _false positive rate_.
- _true positive rate (TPR)_: proportion of true faces that were correctly detected
- _false positive rate (FPR)_: proportion of non-faces that were mis-classified as faces.

Use the below code to calculate the TPR and FPR of your classifiers.

_How does the classifier make errors?_
- As for the False Positive, the class should be Negative, but the model predicts positive. Besides, for the False Negative, the class should be positive, but the model predicts negative.

### Classifier analysis

For the AdaBoost classifier, we can interpret what it is doing by looking at which features it uses most in the weak learners.  Use the below code to visualize the pixel features used.

Note: if you used GridSearchCV to train the classifier, then you need to use the `best_estimator_` field to access the classifier.

In [ ]:
# adacv is the trained adaboost classifier
fi = adacv.best_estimator_.feature_importances_.reshape(imgsize)
plt.imshow(fi, interpolation='nearest')
plt.colorbar()


Similarly, we can also look at the important features for xgboost.

In [ ]:
# xgbcv is the trained xgboost classifier
fi = xgbcv.best_estimator_.feature_importances_.reshape(imgsize)
plt.imshow(fi, interpolation='nearest')
plt.colorbar()


Similarly for Random Forests, we can look at the important features.

In [ ]:
# rfrcv is the trained random forest classifier
fi = rfrcv.best_estimator_.feature_importances_.reshape(imgsize)
plt.imshow(fi, interpolation='nearest')
plt.colorbar()


Comment on which features (pixels) that AdaBoost and Random Forests are using
- As for the Boosting methods, it used the overall features of the image. In specific, as for the face, it used corners, edges, and details of the face. In contrast, the random forest used features from the top to bottom like a tree. In specific, it can learn the features of eyes, nose, etc.

For kernel SVM, we can look at the support vectors to see what the classifier finds difficult.

In [ ]:
# svmclf is the trained SVM classifier
print("num support vectors:", len(svmrbfcv.best_estimator_.support_vectors_))
si  = svmrbfcv.best_estimator_.support_  # get indicies of support vectors

# get all the patches for each support vector
simg = [imgdata['train'][i] for i in si]

# make montage
outimg = image_montage(simg, maxw=20)

plt.figure(figsize=(9,9))
plt.imshow(outimg, cmap='gray', interpolation='nearest')


In [ ]:
# svmclf is the trained SVM classifier
print("num support vectors:", len(svmpolcv.best_estimator_.support_vectors_))
si  = svmpolcv.best_estimator_.support_  # get indicies of support vectors

# get all the patches for each support vector
simg = [imgdata['train'][i] for i in si]

# make montage
outimg = image_montage(simg, maxw=20)

plt.figure(figsize=(9,9))
plt.imshow(outimg, cmap='gray', interpolation='nearest')


Comment on anything you notice about what the SVM finds difficult (i.e., on the decision boundary or within the margin)
- The SVM can capture the overall features from the image. However, those details may be missed, like the details of a specific part of the face.

### Custom kernel SVM

Now we will try to use a custom kernel with the SVM.  We will consider the following RBF-like kernel based on L1 distance (i.e., cityblock or Manhattan distance),

$$ k(\mathbf{x},\mathbf{y}) = \exp \left(-\alpha \sum_{i=1}^d |x_i-y_i|\right)$$

where $x_i,y_i$ are the elements of the vectors $\mathbf{x},\mathbf{y}$, and $\alpha$ is the hyperparameter.  The difference with the RBF kernel is that the new kernel uses the absolute difference rather than the squared difference. Thus, the new kernel does not "drop off" as fast as the RBF kernel using squared distance.

- Implement the new kernel as a custom kernel function. The `scipy.spatial.distance.cdist` function will be helpful.
- Train the SVM with the new kernel.  To select the hyperparameter $\alpha$, you need to run cross-validation "manually" by: 1) trying different values of $\alpha$, and running cross-validation to select $C$; 2) selecting the  $\alpha$ with the highest cross-validation score `best_score_` in `GridSearchCV`.


In [ ]:
from scipy import spatial

def kernel(X1, X2, alpha=1.0):
    "Kernel function"
    
    dis = spatial.distance.cdist(X1, X2, metric='cityblock')
    dis = exp(-alpha * dis)
    
    return dis


paramgrid = {'C': logspace(-4, 4, 3)}
svmmodel = model_selection.GridSearchCV(svm.SVC(kernel=kernel), 
                                      paramgrid, 
                                      cv=5, 
                                      n_jobs=-1, 
                                      verbose=True)

svmmodel.fit(trainXn, trainY);
print("best params:", svmmodel.best_params_)
print("best score:", svmmodel.best_score_)


In [ ]:
alphas = [0.01, 0.05, 0.1, 0.2, 0.5, 1., 10.]


for i in alphas:
    paramgrid = {'C': logspace(-4, 4, 3)}
    svmmodel = model_selection.GridSearchCV(svm.SVC(kernel=kernel), 
                                          paramgrid, 
                                          cv=5, 
                                          n_jobs=-1, 
                                          verbose=True)

    svmmodel.fit(trainXn, trainY);
    print("best params:", svmmodel.best_params_)
    print("best score:", svmmodel.best_score_)


In [ ]:
# predict from the model 
test_pre = svmmodel.predict(testXn)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(SVM Model with more search parameters)\n testing accuracy =", test_acc)


Does using the new kernel improve the results?
- Not really.

## 3. Image Feature Extraction
The detection performance is not that good. The problem is that we are using the raw pixel values as features, so it is difficult for the classifier to interpret larger structures of the face that might be important.  To fix the problem, we will extract features from the image using a set of filters.

Run the below code to look at the filter output.  The filters are a sets of black and white boxes that respond to similar structures in the image.  After applying the filters to the image, the filter response map is aggregated over a 4x4 window.  Hence each filter produces a 5x5 feature response.  Since there are 4 filters, then the feature vector is 100 dimensions.

In [ ]:
def extract_features(imgs, doplot=False):
    # the filter layout
    lay = [array([-1,1]), array([-1,1,-1]),  
               array([[1],[-1]]), array([[-1],[1],[-1]])]
    sc=8            # size of each filter patch
    poolmode = 'i'  # pooling mode (interpolate)
    cmode = 'same'  # convolution mode
    brick = ones((sc,sc))  # filter patch
    ks = []
    for l in lay:
        tmp = [brick*i for i in l]
        if (l.ndim==1):
            k = hstack(tmp)
        else:
            k = vstack(tmp)
        ks.append(k)

    # get the filter response size
    if (poolmode=='max') or (poolmode=='absmax'):
        tmpimg = maxpool(maxpool(imgs[0]))
    else:
        tmpimg = ndimage.interpolation.zoom(imgs[0], 0.25)        
    fs = prod(tmpimg.shape)
    
    # get the total feature length
    fst = fs*len(ks)

    # filter the images
    X  = empty((len(imgs), fst))
    for i,img in enumerate(imgs):
        x = empty(fst)

        # for each filter
        for j,th in enumerate(ks):
            # filter the image
            imgk = signal.convolve(img, ks[j], mode=cmode)
            
            # do pooling
            if poolmode == 'maxabs':
                mimg = maxpool(maxpool(abs(imgk)))
            elif poolmode == 'max':
                mimg = maxpool(maxpool(imgk))
            else:
                mimg = ndimage.interpolation.zoom(imgk, 0.25)
    
            # put responses into feature vector
            x[(j*fs):(j+1)*fs] = ravel(mimg)
               
            if (doplot):             
                plt.subplot(3,len(ks),j+1)
                plt.imshow(ks[j], cmap='gray', interpolation='nearest')
                plt.title("filter " + str(j))
                plt.subplot(3,len(ks),len(ks)+j+1)
                plt.imshow(imgk, cmap='gray', interpolation='nearest')
                plt.title("filtered image")
                plt.subplot(3,len(ks),2*len(ks)+j+1)
                plt.imshow(mimg, cmap='gray', interpolation='nearest')
                plt.title("image features")
        X[i,:] = x
    
    return X

In [ ]:
# new features
img = imgdata['train'][0]
plt.imshow(img, cmap='gray', interpolation='nearest')
plt.title("image")
plt.figure(figsize=(9,9))
extract_features([img], doplot=True)


Now lets extract image features on the training and test sets.  It may take a few seconds.

In [ ]:
trainXf = extract_features(imgdata['train'])
print(trainXf.shape)

testXf = extract_features(imgdata['test'])
print(testXf.shape)


### Detection using Image Features
Now train AdaBoost and SVM classifiers on the image feature data.  Evaluate on the test set.

In [ ]:
# first scale the features
scalerf = preprocessing.MinMaxScaler(feature_range=(-1,1))    # make scaling object
trainXfn = scalerf.fit_transform(trainXf)   # use training data to fit scaling parameters
testXfn  = scalerf.transform(testXf)        # apply scaling to test data


In [ ]:
# AdaBoost Model
# setup the list of parameters to try
paramgrid = {'learning_rate': logspace(-6, 0, 3)} 
# print(paramgrid)

# setup the cross-validation object 
adaclf = ensemble.AdaBoostClassifier(random_state=4487)
adacv = model_selection.GridSearchCV(adaclf, paramgrid, cv=5, n_jobs=8) 

adacv.fit(trainXfn, trainY)

# predict from the model 
train_pre = adacv.predict(trainXfn)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(AdaBoost Model)\n training accuracy =", train_acc)

# predict from the model 
test_pre = adacv.predict(testXfn)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(AdaBoost Model)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


In [ ]:
# SVM Model with RBF kernel
# setup the list of parameters to try 
paramgrid = {'C': logspace(-2, 3, 3), 
             'gamma': logspace(-4, 3, 3)}
print(paramgrid)

# setup the cross-validation object 
# pass the SVM object w/ rbf kernel, parameter grid, and number of CV folds 
svmrbf = svm.SVC(kernel='rbf')
svmrbfcv = model_selection.GridSearchCV(svmrbf, 
                                     paramgrid, 
                                     cv=5, 
                                     n_jobs=8, 
                                     verbose=True)

# run cross-validation (train for each split) 
svmrbfcv.fit(trainXfn, trainY);
print("best params:", svmcv.best_params_)

# predict from the model 
train_pre = svmrbfcv.predict(trainXfn)
train_acc = metrics.accuracy_score(trainY, train_pre) 
print("(SVM Model with RBF kernel with more search parameters)\n training accuracy =", train_acc)

# predict from the model 
test_pre = svmrbfcv.predict(testXfn)
test_acc = metrics.accuracy_score(testY, test_pre) 
print("(SVM Model with RBF kernel with more search parameters)\n testing accuracy =", test_acc)

# predY is the prediction from the classifier
predY = test_pre
Pind = where(predY == 1) # indicies for face predictions
Nind = where(predY == 0) # indicies for non-face predictions
TP = count_nonzero(testY[Pind] == predY[Pind])
FP = count_nonzero(testY[Pind] != predY[Pind])
TN = count_nonzero(testY[Nind] == predY[Nind])
FN = count_nonzero(testY[Nind] != predY[Nind])

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

print("TP =", TP)
print("FP =", FP)
print("TN =", TN)
print("FN =", FN)
print("TPR=", TPR)
print("FPR =", FPR)


### Error Analysis
Repeat the error analysis for the new classifiers.

In [ ]:
# The Confusion Matrix info has added into the above codes.

How has the classifier using image features improved?
- Yes! It seems that such features are quite helpful!

# Test image
Now lets try your face detector on a real image.  Download the "nasa-small.png" image and put it in the same directory as your ipynb file.  The below code will load the image, crop out image patches and then extract features. (this may take a few minutes)

In [ ]:
fname = "nasa-small.png"

In [ ]:
# load image
testimg3 = skimage.io.imread(fname)

# convert to grayscale
testimg = skimage.color.rgb2gray(testimg3)
print(testimg.shape)
plt.imshow(testimg, cmap='gray')

In [ ]:
# step size for the sliding window
step = 4

# extract window patches with step size of 4
patches = skimage.util.view_as_windows(testimg, (19,19), step=step)
psize = patches.shape
# collapse the first 2 dimensions
patches2 = patches.reshape((psize[0]*psize[1], psize[2], psize[3]))
print(patches2.shape )

# histogram equalize patches (improves contrast)
patches3 = empty(patches2.shape)
for i in range(patches2.shape[0]):
    patches3[i,:,:] = skimage.exposure.equalize_hist(patches2[i,:,:])

# extract features
newXf = extract_features(patches3)

Now predict using your classifier.  The extracted features are in `newXf`, and scaled features are `newXfn`.

In [ ]:
newXfn  = scalerf.transform(newXf)        # apply scaling to test data

In [ ]:
# predict from the Adaboost model 
prednewY = adacv.predict(newXfn)
print("(Adaboost with more search parameters)\n Prediction: ", prednewY)


In [ ]:
# reshape prediction to an image
imgY = prednewY.reshape(psize[0], psize[1])

# zoom back to image size
imgY2 = ndimage.interpolation.zoom(imgY, step, output=None, order=0)
# pad the top and left with half the window size
imgY2 = vstack((zeros((9, imgY2.shape[1])), imgY2))
imgY2 = hstack((zeros((imgY2.shape[0],9)), imgY2))
# pad right and bottom to same size as image
if (imgY2.shape[0] != testimg.shape[0]):
    imgY2 = vstack((imgY2, zeros((testimg.shape[0]-imgY2.shape[0], imgY2.shape[1]))))
if (imgY2.shape[1] != testimg.shape[1]):
    imgY2 = hstack((imgY2, zeros((imgY2.shape[0],testimg.shape[1]-imgY2.shape[1]))))
    
# show detections with image
#detimg = dstack(((0.5*imgY2+0.5)*testimg, 0.5*testimg, 0.5*testimg))
nimgY2 = 1-imgY2
tmp = nimgY2*testimg
detimg = dstack((imgY2+tmp, tmp, tmp))

# show it!
plt.figure(figsize=(9,9))
plt.subplot(2,1,1)
plt.imshow(imgY2, interpolation='nearest')
plt.title('detection map')
plt.subplot(2,1,2)
plt.imshow(detimg)
plt.title('image')
plt.axis('image')

In [ ]:
# predict from the SVM Model with RBF kernel model 
prednewY = svmrbfcv.predict(newXfn)
print("(SVM Model with RBF kernel with more search parameters)\n Prediction: ", prednewY)


Now we we will view the results on the image.  Use the below code. `prednewY` is the vector of predictions.

In [ ]:
# reshape prediction to an image
imgY = prednewY.reshape(psize[0], psize[1])

# zoom back to image size
imgY2 = ndimage.interpolation.zoom(imgY, step, output=None, order=0)
# pad the top and left with half the window size
imgY2 = vstack((zeros((9, imgY2.shape[1])), imgY2))
imgY2 = hstack((zeros((imgY2.shape[0],9)), imgY2))
# pad right and bottom to same size as image
if (imgY2.shape[0] != testimg.shape[0]):
    imgY2 = vstack((imgY2, zeros((testimg.shape[0]-imgY2.shape[0], imgY2.shape[1]))))
if (imgY2.shape[1] != testimg.shape[1]):
    imgY2 = hstack((imgY2, zeros((imgY2.shape[0],testimg.shape[1]-imgY2.shape[1]))))
    
# show detections with image
#detimg = dstack(((0.5*imgY2+0.5)*testimg, 0.5*testimg, 0.5*testimg))
nimgY2 = 1-imgY2
tmp = nimgY2*testimg
detimg = dstack((imgY2+tmp, tmp, tmp))

# show it!
plt.figure(figsize=(9,9))
plt.subplot(2,1,1)
plt.imshow(imgY2, interpolation='nearest')
plt.title('detection map')
plt.subplot(2,1,2)
plt.imshow(detimg)
plt.title('image')
plt.axis('image')

_How did your face detector do?_
- Classify each block into 'human face' or 'not human face'. And visualize the results. 

You can try it on your own images.  The faces should all be around 19x19 pixels though.
We only used 1/4 of the training data. Try using more data to train it!